In [43]:
import numpy as np
import pandas as pd
import networkx as nx
import traceback
import os
import matplotlib.pyplot as plt
import math
from ant_defs import Ant, QapAnt
import time
import tsplib95
from ant_defs import observer
from ant_defs import makestatsfile
from random import randint

In [44]:
def findbest(pop: list):
    best = Ant(0)
    best.setTourLength(10000000000000.0)
    for ant in pop:
        if ant.isTourViable():
            if ant < best:
                best = ant
    return best

In [45]:
from QAPInstances import getBur26a
from QAPInstances import getnug15
from QAPInstances import getWill100
from QAPInstances import getTai60a
matD, matF = getWill100()
matD = np.matrix(matD)
matF = np.matrix(matF)
vectA = matD.sum(axis=1)
vectB = matF.sum(axis=1)
graphweights = np.dot(vectA,vectB.T)
graph = nx.from_numpy_matrix(graphweights, create_using=nx.DiGraph)
nx.set_edge_attributes(graph,1.0,"pheromone")
nx.set_edge_attributes(graph,'b',"color")
nx.set_edge_attributes(graph,0.0,"heuristic")
nx.set_edge_attributes(graph,0.0,"delta")
# at = QapAnt(0)
# test = [0,1,12,7,8,3,2,13,6,10,9,14,5,4,11]
# s = 0
# for element in test:
#     at.move(element,nugd,nugf)
# print(at.getTourLength())
# for i in range(0,15):
#     for j in range(0,15):
#         x = test[i]
#         y = test[j]
#         s = s + nugd.item((i,j))*nugf.item((x,y))
# print(s)
for u, v, weight in graph.edges(data="weight"):
    if(weight is not None and weight != 0):
        #print(f'source {u}, dest {v}, weight {weight}')
        graph[u][v]["heuristic"] = 1/weight

==============================================================================

ANT SYSTEM

======================================================================================

In [46]:
def ASNextNodeSelection(graph: nx.DiGraph,viable_paths: list,visited: list,args: dict):
    psum = 0
    pvals = []
    for node in viable_paths:
        tmp = math.pow(graph[visited[-1]][node]["heuristic"],args['heuristic_preference']) + math.pow(graph[visited[-1]][node]["pheromone"],args['pheromone_preference'])
        pvals.append(tmp)
    s = np.sum(pvals)
    probs = [x/s for x in pvals]
    next_node = np.random.choice(a=viable_paths,p=probs,replace=False)
    return next_node

def ASGlobalUpdate(graph: nx.DiGraph,population: list, args: dict) -> nx.DiGraph:
    for ant in population:
        #proper solutions only
        if ant.isTourViable(): 
            tour = ant.getTour()
            startnodes = tour[:-1]
            endnodes = tour[1:]
            for s,n in zip(startnodes,endnodes):
                graph[s][n]["delta"] = graph[s][n]["delta"] + (args['Qval'] / ant.getTourLength())
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"]
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

In [47]:
def AntSystemTSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = QapAnt(1)
    globalbest.setTourLength(100000000000.0)
    argslocal['globalbest'] = globalbest
    localbest = QapAnt(1)
    localbest.setTourLength(100000000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            startnode = randint(0,len(matD)-1)
            population.append(QapAnt(startnode))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        print("ant invalidated")
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,matD,matF)
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        #post cycle pheromone updates and solution assesment
        #review population
        poptours = [x.getTourLength() for x in population if x.isTourViable()]
        localbest = findbest(population)
        #print(localbest.getTourLength())
        #print(consecutive_same_best)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 15:
            stagnation = i
            break
        problem = ASGlobalUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'AS', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

ANT COLONY ACO

=====================================================================================

In [48]:
def ACONextNodeSelection(graph: nx.DiGraph,viable_paths: list,visited: list,args: dict):
    pvals = []
    if args['selectionTypeConst'] > np.random.uniform():
        for node in viable_paths:
            tmp = graph[visited[-1]][node]["heuristic"] + math.pow(graph[visited[-1]][node]["pheromone"],args['heuristic_preference'])
            pvals.append(tmp)
        return viable_paths[np.argmax(pvals)]
    else:
        return ASNextNodeSelection(graph,viable_paths,visited,args)

def ACOGlobalPheromoneUpdate(graph: nx.DiGraph,population: list, args: dict):
    evap_coef = 1-args['evaporation']
    best = args['globalbest']
    tour = best.getTour()
    startnodes = tour[:-1]
    endnodes = tour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["delta"] += math.pow(best.getTourLength(),-1)
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = (evap_coef*graph[u][v]["pheromone"]) + (args['evaporation']*graph[u][v]["delta"])
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

def ACOLocalUpdateTau0(graph: nx.DiGraph, lastnode, newnode, taboo: list, args: dict):
    evap_coef = 1-args['evaporation']
    one = evap_coef*graph[lastnode][newnode]['pheromone']
    two = args['evaporation']*args['startpheromone']
    graph[lastnode][newnode]["pheromone"] = one + two
    return graph

def ACOLocalUpdateQlearning(graph: nx.DiGraph, lastnode, newnode, taboo: list, args: dict):
    evap_coef = 1-args['evaporation']
    nextnodes = [x for x in [n for n in graph[newnode]] if x not in taboo]
    if len(nextnodes) == 0:
        return ACOLocalUpdateTau0(graph,lastnode,newnode,taboo,args)
    else:
        nextphero = []
        for node in nextnodes:
            nextphero.append(graph[newnode][node]['pheromone'])
        delta = args['ACS_qlearn_gamma']*np.max(nextphero)
        one = evap_coef*graph[lastnode][newnode]['pheromone']
        two = args['evaporation']*delta
        graph[lastnode][newnode]["pheromone"] = one + two
        return graph

In [49]:
def ACO_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = QapAnt(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = QapAnt(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            startnode = randint(0,len(matD)-1)
            population.append(QapAnt(startnode))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ACONextNodeSelection(problem,viable_paths,visited,argslocal)
                    ACOLocalUpdateQlearning(problem,visited[-1],next_node,visited,argslocal)
                    ant.move(next_node,matD,matF)
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        # for ant in population:
        #     visited = ant.getTour()
        #     ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        #post cycle pheromone updates and solution assesment
        #review population
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 15:
            stagnation = i
            break
        problem = ACOGlobalPheromoneUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'ACO', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

MMAS

=====================================================================================

In [50]:
def MMAStaumax(graph: nx.DiGraph, bestlength: float, args: dict):
    one = 1/(1-args['evaporation'])
    two = 1/bestlength
    max_pheromone = one * two
    args['max_pheromone'] = max_pheromone
    return args

def MMAStaumin(graph: nx.DiGraph, args: dict):
    n = graph.number_of_nodes()
    p = args['best_pheromone_selection_prob']**(1/n)
    av = n/2
    min_pheromone = (args['max_pheromone']*(1-p))/((av-1)*p)
    args['min_pheromone'] = min_pheromone
    return args

def MMASTrailSmoothing(graph: nx.DiGraph, args: dict):
    for u, v, pheromone in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = pheromone + args['smoothing_coef']*(args['max_pheromone']-pheromone)
    return graph

def MMASGlobalUpdate(graph: nx.DiGraph,localbest, args: dict) -> nx.DiGraph:
        #proper solutions only
    tour = localbest.getTour()
    startnodes = tour[:-1]
    endnodes = tour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["delta"] += args['Qval'] / localbest.getTourLength()
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        new_pheromone = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"]
        if new_pheromone > args['max_pheromone']:
            new_pheromone = args['max_pheromone']
        elif new_pheromone < args['min_pheromone']:
            new_pheromone = args['min_pheromone']
        graph[u][v]["pheromone"] = new_pheromone
    nx.set_edge_attributes(graph,0.0,"delta")
    return graph

In [51]:
def MMAS_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int,stats_file, args: dict) -> Ant:
    start = time.time()
    max_stagnations = 2
    current_stagnations = 0
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    population = []
    globalbest = QapAnt(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = QapAnt(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            startnode = randint(0,len(matD)-1)
            population.append(QapAnt(startnode))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,matD,matF)
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        #post cycle pheromone updates and solution assesment
        #review population
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        argslocal = MMAStaumax(problem,globalbest.getTourLength(),argslocal)
        argslocal = MMAStaumin(problem,argslocal)
        problem = MMASGlobalUpdate(problem,localbest,argslocal)
        poptours = [x.getTourLength() for x in population if x.isTourViable()]
        if consecutive_same_best > 15:
            stagnation = i
            problem = MMASTrailSmoothing(problem,argslocal)
            consecutive_same_best = 0
            current_stagnations = current_stagnations + 1
        if current_stagnations > max_stagnations:
            break
        # TODO: Find way to detect convergence and stagnation to insert trail smoothing here
    end = time.time()
    extime = end - start
    observer(stats_file,'MMAS', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

=====================================================================================

ASRANK

=====================================================================================

In [52]:
def ASRANKGlobalUpdate(graph,population,args):
    good_solutions = [x for x in population if x.isTourViable()]
    good_solutions.sort(reverse=True)
    ranks = args['elite_multiplier']-1
    used_solutions = good_solutions[-ranks:]

    for rank, ant in enumerate(used_solutions):
        r_rank = rank + 1
        tour = ant.getTour()
        startnodes = tour[:-1]
        endnodes = tour[1:]
        for s,n in zip(startnodes,endnodes):
            graph[s][n]["delta"] += (args['Qval'] / ant.getTourLength())*r_rank
    best = args['globalbest']   
    besttour = best.getTour()
    startnodes = besttour[:-1]
    endnodes = besttour[1:]
    for s,n in zip(startnodes,endnodes):
        graph[s][n]["elite_delta"] += (args['Qval'] / best.getTourLength())*args['elite_multiplier']
    evap_coef = 1-args['evaporation']
    for u, v, weight in graph.edges(data="pheromone"):
        graph[u][v]["pheromone"] = evap_coef*graph[u][v]["pheromone"] + graph[u][v]["delta"] + graph[u][v]["elite_delta"]
    nx.set_edge_attributes(graph,0.0,"delta")
    nx.set_edge_attributes(graph,0.0,"elite_delta")
    return graph

In [53]:
def ASRANK_TSP(problem: nx.DiGraph, pop_count: int, max_cycles: int, stats_file, args: dict) -> Ant:
    start = time.time()
    best_solution_cycle = -1
    stagnation = 0
    consecutive_same_best = 0
    argslocal = args
    problem = problem
    nx.set_edge_attributes(problem,args['startpheromone'],"pheromone")
    nx.set_edge_attributes(problem,0.0,"elite_delta")
    population = []
    globalbest = QapAnt(0)
    globalbest.setTourLength(100000000.0)
    argslocal['globalbest'] = globalbest
    localbest = QapAnt(0)
    localbest.setTourLength(100000000.0)
    for i in range(0,max_cycles):
        population = []
        for k in range(0,pop_count):
            startnode = randint(0,len(matD)-1)
            population.append(QapAnt(startnode))
        for j in range(0,len(problem)-1):
            for ant in population:
                visited = ant.getTour()
                # i forgot wtf is going on here
                # ok so we assume the last visited node is our starting node
                # and NX Graph gives us a list of all its neighbours
                # and we check if we were already there
                viable_paths = [x for x in [n for n in problem[visited[-1]]] if x not in visited]
                if len(viable_paths) == 0:
                    if j < len(problem)-1:
                        ant.invalidateTour()
                    pass
                else:
                    next_node = ASNextNodeSelection(problem,viable_paths,visited,argslocal)
                    ant.move(next_node,matD,matF)
                #ants have moved 1 node ahead
            #most ants now have a solution ready
        #post cycle pheromone updates and solution assesment
        #review population
        # for ant in population:
        #     visited = ant.getTour()
        #     ant.move(visited[0],problem[visited[-1]][visited[0]]["weight"])
        localbest = findbest(population)
        if localbest < globalbest:
            globalbest = localbest
            argslocal['globalbest'] = globalbest
            best_solution_cycle = i
            consecutive_same_best = 0
        else:
            consecutive_same_best = consecutive_same_best + 1
        if consecutive_same_best > 15:
            stagnation = i
            break
        problem = ASRANKGlobalUpdate(problem,population,argslocal)
    end = time.time()
    extime = end - start
    observer(stats_file,'ASRANK', pop_count, stagnation, best_solution_cycle,globalbest, extime, argslocal)
    return problem, globalbest

In [54]:
stats_file = makestatsfile('Will100.csv')

In [55]:
# p = tsplib95.load('ft53.atsp')
# p.edge_weights  
# graph = p.get_graph()
# nx.set_edge_attributes(graph,1.0,"pheromone")
# nx.set_edge_attributes(graph,'b',"color")
# nx.set_edge_attributes(graph,0.0,"heuristic")
# nx.set_edge_attributes(graph,0.0,"delta")

# for u, v, weight in graph.edges(data="weight"):
#     if(weight is not None and weight != 0):
#         print(f'source {u}, dest {v}, weight {weight}')
#         graph[u][v]["heuristic"] = 1/weight

In [56]:
args_AS = {
    'pheromone_preference': 1,
    'heuristic_preference': 1,
    'Qval': 2,
    'evaporation': 0.5,
    'startpheromone': 2.0
}

args_ACO = {
    'pheromone_preference': 1,
    'heuristic_preference': 1,
    'Qval': 1,
    'evaporation': 0.9,
    'startpheromone': 2.0,
    'selectionTypeConst': 0.3,
    'ACS_qlearn_gamma': 0.5
}

args_MMAS = {
    'pheromone_preference': 10,
    'heuristic_preference': 5,
    'Qval': 1,
    'evaporation': 0.9,
    'startpheromone': 2.0,
    'best_pheromone_selection_prob': 0.05,
    'max_pheromone': 4.0,
    'min_pheromone': 0.5,
    'smoothing_coef': 0.5
}

args_ASRANK = {
    'pheromone_preference': 10,
    'heuristic_preference': 5,
    'Qval': 1,
    'evaporation': 0.9,
    'startpheromone': 1.0,
    'elite_multiplier': 25
}

# args_AS = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0
# }

# args_ACO = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'selectionTypeConst': 0.5,
#     'ACS_qlearn_gamma': 0.5
# }

# args_MMAS = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'best_pheromone_selection_prob': 0.05,
#     'max_pheromone': 2.0,
#     'min_pheromone': 0.5,
#     'smoothing_coef': 0.5
# }

# args_ASRANK = {
#     'pheromone_preference': 1,
#     'heuristic_preference': 1,
#     'Qval': 1,
#     'evaporation': 0.5,
#     'startpheromone': 1.0,
#     'elite_multiplier': 62
# }
# kro124p - 36230
# ft53 - 6905
# ft70 - 1950
# ftv35 - 1473

In [57]:
for d in range(0,3):
    G, solution = AntSystemTSP(graph,26,1000,stats_file,args_AS)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[54, 1, 5, 4, 85, 83, 19, 24, 97, 47, 76, 81, 36, 38, 69, 12, 92, 90, 44, 99, 56, 7, 41, 70, 28, 88, 15, 78, 3, 10, 55, 95, 46, 89, 48, 61, 75, 29, 27, 66, 77, 43, 49, 26, 34, 60, 11, 16, 84, 80, 45, 50, 9, 98, 93, 64, 58, 31, 62, 37, 20, 22, 57, 72, 51, 91, 52, 6, 2, 13, 35, 53, 17, 79, 68, 73, 87, 18, 82, 32, 63, 8, 42, 21, 40, 25, 23, 30, 67, 0, 65, 86, 33, 71, 94, 59, 39, 74, 96, 14]
295678
[56, 95, 87, 23, 72, 46, 21, 85, 18, 91, 88, 89, 96, 4, 42, 61, 70, 36, 7, 32, 11, 54, 81, 2, 62, 76, 80, 1, 8, 14, 79, 12, 25, 93, 58, 98, 63, 40, 51, 37, 83, 31, 82, 24, 74, 33, 22, 20, 59, 68, 47, 38, 73, 90, 45, 94, 77, 75, 97, 5, 3, 39, 41, 28, 78, 65, 57, 43, 67, 13, 29, 30, 10, 26, 34, 55, 53, 71, 50, 27, 16, 86, 48, 52, 19, 9, 15, 44, 66, 69, 64, 60, 84, 6, 92, 49, 0, 99, 35, 17]
296222
[48, 13, 32, 3, 8, 86, 24, 63, 31, 96, 62, 72, 16, 17, 12, 28, 70, 80, 49, 68, 37, 54, 46, 0, 99, 44, 90, 75, 7, 87, 9, 10, 82, 39, 18, 57, 59, 66, 98, 29, 77, 45, 25, 85, 51, 83, 71, 33, 41, 27, 65, 11, 

In [36]:
for d in range(0,3):
    graph, solution = ACO_TSP(graph,53,1000,stats_file,args_ACO)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[14, 32, 49, 3, 88, 17, 24, 73, 62, 70, 91, 0, 16, 56, 50, 96, 43, 20, 69, 15, 33, 67, 84, 47, 34, 89, 4, 85, 44, 86, 61, 29, 59, 79, 40, 28, 31, 6, 22, 35, 93, 30, 1, 8, 12, 41, 27, 48, 77, 98, 23, 87, 90, 9, 65, 63, 83, 64, 21, 66, 55, 92, 45, 94, 38, 71, 53, 54, 74, 42, 72, 80, 39, 76, 81, 10, 68, 18, 99, 7, 5, 78, 19, 52, 97, 13, 11, 36, 26, 57, 2, 60, 58, 95, 75, 82, 46, 51, 25, 37]
296342
[22, 82, 32, 27, 5, 52, 14, 9, 49, 31, 91, 80, 88, 0, 84, 98, 67, 94, 96, 86, 56, 47, 8, 50, 26, 62, 89, 30, 63, 61, 28, 54, 11, 48, 51, 23, 40, 59, 92, 81, 93, 38, 85, 29, 97, 15, 20, 79, 65, 64, 60, 69, 1, 66, 71, 87, 6, 33, 39, 16, 58, 10, 18, 19, 68, 99, 41, 13, 77, 43, 2, 3, 34, 83, 24, 90, 70, 7, 75, 12, 42, 45, 4, 35, 78, 21, 95, 72, 73, 17, 76, 74, 46, 57, 37, 36, 55, 25, 44, 53]
295240
[49, 32, 14, 91, 25, 79, 0, 98, 56, 40, 26, 86, 47, 97, 62, 78, 38, 22, 61, 18, 48, 28, 92, 31, 20, 93, 44, 73, 84, 33, 8, 27, 3, 36, 43, 96, 4, 42, 88, 72, 76, 29, 35, 37, 15, 89, 94, 9, 70, 65, 63, 64, 

In [37]:
for d in range(0,3):
    graph, solution = MMAS_TSP(graph,53,1000,stats_file,args_MMAS)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[75, 49, 31, 8, 44, 0, 45, 28, 62, 3, 32, 24, 48, 14, 83, 61, 42, 5, 73, 59, 35, 67, 87, 54, 15, 38, 77, 72, 98, 21, 97, 91, 22, 56, 12, 74, 96, 20, 4, 23, 81, 41, 66, 13, 94, 19, 78, 27, 93, 29, 6, 47, 71, 2, 55, 17, 64, 79, 50, 51, 60, 57, 58, 9, 70, 68, 86, 95, 46, 43, 39, 65, 16, 76, 88, 25, 34, 99, 80, 63, 85, 36, 11, 82, 84, 37, 33, 26, 1, 30, 90, 89, 52, 92, 7, 10, 69, 18, 53, 40]
294934
[75, 94, 88, 56, 73, 95, 0, 91, 49, 99, 80, 16, 41, 8, 69, 34, 93, 38, 87, 33, 14, 11, 76, 40, 5, 59, 3, 85, 50, 32, 83, 51, 42, 67, 82, 77, 55, 45, 47, 89, 28, 23, 65, 15, 13, 54, 98, 79, 58, 62, 26, 61, 96, 19, 20, 22, 17, 52, 48, 57, 39, 81, 70, 21, 31, 30, 71, 97, 18, 74, 1, 84, 7, 12, 10, 6, 63, 2, 64, 78, 60, 68, 9, 29, 35, 92, 46, 66, 90, 27, 24, 43, 25, 4, 36, 72, 53, 86, 44, 37]
295360
[34, 27, 66, 1, 14, 30, 94, 84, 10, 95, 16, 37, 48, 62, 32, 29, 56, 86, 55, 64, 45, 87, 44, 85, 6, 47, 81, 63, 91, 69, 43, 28, 70, 19, 59, 58, 42, 24, 15, 49, 20, 68, 22, 88, 76, 11, 60, 33, 13, 35, 80, 6

In [38]:
for d in range(0,3):
    graph, solution = ASRANK_TSP(graph,53,1000,stats_file,args_ASRANK)
    besttour = solution.getTour()
    print(besttour)
    print(solution.getTourLength())

[91, 49, 11, 51, 98, 18, 54, 63, 56, 5, 15, 0, 24, 27, 94, 14, 31, 77, 37, 42, 26, 70, 17, 28, 9, 53, 66, 99, 81, 48, 32, 84, 35, 50, 67, 75, 39, 52, 64, 30, 40, 68, 8, 76, 33, 2, 10, 38, 78, 21, 88, 74, 3, 97, 1, 90, 29, 87, 4, 95, 6, 12, 83, 92, 55, 96, 47, 79, 36, 57, 62, 43, 41, 7, 73, 22, 89, 25, 58, 20, 86, 44, 61, 85, 69, 93, 80, 60, 45, 23, 82, 13, 59, 19, 65, 16, 71, 34, 72, 46]
295840
[13, 62, 14, 15, 66, 63, 3, 9, 49, 0, 81, 93, 28, 34, 29, 47, 48, 94, 50, 59, 80, 12, 55, 91, 31, 90, 43, 71, 84, 77, 70, 85, 46, 23, 56, 1, 58, 87, 4, 17, 8, 25, 92, 65, 51, 75, 99, 21, 76, 6, 61, 69, 2, 19, 30, 32, 24, 60, 68, 33, 54, 95, 89, 20, 36, 35, 40, 22, 45, 73, 64, 72, 26, 74, 86, 5, 38, 16, 18, 98, 88, 82, 7, 39, 78, 96, 44, 11, 83, 37, 79, 67, 10, 41, 53, 57, 27, 52, 42, 97]
295404


KeyboardInterrupt: 